In [9]:
import pandas as pd
import re
import torch
import torch.nn as nn
from collections import Counter
from IPython.core.debugger import set_trace
import csv
import string
import unidecode
import torch.nn.functional as F

In [10]:
def removeLink(s):
    pattern = r'http\S+'
    return re.sub(pattern,'', s)

In [11]:
def readFiles(file_name):
    output_array = list()
    with open(file_name,'r', encoding='UTF-8', newline='') as file:
        reader=csv.reader(file)
        for i, row in enumerate(reader):
            if i > 0:
                line = removeLink(row[-1][2:])
                line = re.sub(' +',' ', line)
                line = unidecode.unidecode(line)                
                output_array.append(line)
        
    file.close()
    
    return output_array

output_array = readFiles('trumpTweets.csv')
#[string.printable.index(c) for c in output_array[0]]

In [14]:
def char_to_tensor(tA):
    #set_trace()
    lst = [string.printable.index(c) for c in tA]
    tensorString = torch.tensor(lst).long()
    return tensorString

def getInputOutput(textArray, index_data=0):
    text_long = char_to_tensor(textArray[index_data])
    input_line = text_long[:-1]
    output_line = text_long[1:]
    return input_line, output_line

In [ ]:
RANDOM_SEED = 123
torch.manual_seed(RANDOM_SEED)

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

TEXT_PORTION_SIZE = 200

NUM_ITER = 20000
LEARNING_RATE = 0.005
EMBEDDING_DIM = 100
HIDDEN_DIM = 100
NUM_HIDDEN = 1
LEN_TRAINING_DATA = len(output_array)

In [ ]:
class Network(nn.Module):
    def __init__(self, input_size, embed_size, hidden_size, output_size, num_layers):
        super().__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size        
        self.embed = nn.Embedding(input_size, embed_size)
        self.gru = nn.GRU(embed_size, hidden_size,num_layers=num_layers)
        self.dense = nn.Linear(hidden_size, output_size)
        self.init_hidden = torch.nn.Parameter(torch.zeros(num_layers, 1, hidden_size))
        
    def forward(self, features, hidden):
        embed_feature = self.embed(features.view(1,-1))
        output, hidden = self.gru(embed_feature.view(1,1,-1), hidden)
        output = self.dense(output.view(1,-1))
        return output
    
    def init_zero_state(self):
        init_hidden = torch.zeros(self.num_layers, 1, self.hidden_size).to(DEVICE)
        return init_hidden

In [ ]:
torch.manual_seed(RANDOM_SEED)
model = Network(len(string.printable), EMBEDDING_DIM, HIDDEN_DIM, len(string.printable), NUM_HIDDEN)
model = model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)



In [ ]:
for iteration in range(NUM_ITER):    
    ### FORWARD AND BACK PROP
    hidden = model.init_zero_state()
    optimizer.zero_grad()    
    loss = 0.
    inputs, targets = getInputOutput(output_array)
    set_trace()
    inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
    for c in range(TEXT_PORTION_SIZE):
        outputs, hidden = model(inputs, hidden)
        loss += F.cross_entropy(outputs, targets[c].view(1))

    loss /= TEXT_PORTION_SIZE
    loss.backward()
    
    ### UPDATE MODEL PARAMETERS
    optimizer.step()

    ### LOGGING
    with torch.no_grad():
        if iteration % 1000 == 0:  
            print(f'Iteration {iteration} | Loss {loss.item():.2f}\n\n')
            print(evaluate(model, 'Th', 200), '\n')
            print(50*'=')

In [ ]:
len(string.printable)

In [ ]:
string.printable

In [15]:
inputs = [torch.randn(1, 3) for _ in range(5)] 

In [16]:
inputs

[tensor([[ 1.4783, -0.1375, -1.4389]]),
 tensor([[ 0.0603, -0.4699, -0.2681]]),
 tensor([[ 1.2553, -2.5328,  0.4212]]),
 tensor([[-0.9199,  0.5817, -0.8611]]),
 tensor([[-0.7232,  0.2927,  0.7804]])]

In [23]:
torch.randn(1,3).size()


torch.Size([1, 3])

In [24]:
inputs[0].view(1,1,-1)

tensor([[[ 1.4783, -0.1375, -1.4389]]])